In [ ]:
from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine, text

def open_cra(path):

    cra_columns = pd.read_csv(Path.cwd() / "conf" / "cra_columns.csv")

    michigan = []
    for chunk in pd.read_fwf(
        path, 
        names=cra_columns["column_name"], 
        widths=cra_columns["width"], 
        chunksize=1000
    ):

        michigan.append(chunk[
            chunk["state"] == 26
        ].copy())

    return pd.concat(michigan)


def calc_totals(frame):
    frame["total_count"] = frame[["num_under_100k", "num_100k_to_250k",
        "num_250k_million", "num_over_million"]].sum(axis=1)

    # The raw data is by 1/1000
    frame["total_amt"] = 1000 * frame[["amt_under_100k", "amt_100k_to_250k",
        "amt_250k_million", "amt_over_million"]].sum(axis=1)

    return frame


def process_tract_geoid(tract: pd.Series):
    tract_splits = (
        tract.astype("str")
        .str.split(".", expand=True)
    )

    return (
        tract_splits[0].str.pad(4, side="left", fillchar="0")
        + tract_splits[1].str.pad(2, side="right", fillchar="0")
    )


def short_geoid(frame):
    return (
        frame["state"].astype(pd.Int64Dtype()).astype("str")
        + frame["county"].astype(pd.Int64Dtype()).astype("str").str.pad(3, side="left", fillchar='0')
        + process_tract_geoid(frame["tract"])
    )


In [29]:
population = pd.read_excel("mi_tract_population_20250811.xlsx")
population["short_geoid"] = population["geoid"].str.slice(9)


def prepare_cra_year(year: str):
    short_year = year[2:]
    path = f"{short_year}exp_aggr/cra{year}_Aggr_A11.dat"
    pop = population[population["Year"].astype(str) == year]

    frame = open_cra(path)
    frame = frame[~frame["tract"].isna()].copy() # tracts only
    frame["short_geoid"] = short_geoid(frame)
    frame = calc_totals(frame)

    merged = (
        frame
        .merge(pop, on="short_geoid")
        .sort_values("county")
    )

    # If any rows are lost, beak
    assert len(merged) == len(frame)

    merged["num_cra_loans_per_10k"] = (10_000 * merged["total_count"]) / merged["Total Population"]
    merged["cra_dollars_per_capita"] = (1000 * merged["total_amt"]) / merged["Total Population"]

    return merged

In [30]:
frame = prepare_cra_year('2023')
len(frame)

2939

In [32]:
totals = frame[["Total Population", "total_count", "total_amt"]].sum()

print("Michigan")
print("CRA Loans Per 10k People")
print(10_000 * totals["total_count"] / totals["Total Population"])
print("CRA Loan Amount per Capita")
print(totals["total_amt"] / totals["Total Population"])

Michigan
CRA Loans Per 10k People
267.45947938369784
CRA Loan Amount per Capita
798.2986966971206


In [33]:
totals = frame[
    frame["short_geoid"].str.startswith("26163")
][["Total Population", "total_count", "total_amt"]].sum()

print("Wayne County")
print("CRA Loans Per 10k People")
print(10_000 * totals["total_count"] / totals["Total Population"])
print("CRA Loan Amount per Capita")
print(1_000 * totals["total_amt"] / totals["Total Population"])

Wayne County
CRA Loans Per 10k People
289.2126103666419
CRA Loan Amount per Capita
699299.8437770437


In [35]:
db = create_engine("postgresql+psycopg://mike:a norse horse of course@edw:5432/data")

(
    frame[["short_geoid", "total_count", "total_amt"]]
    .rename(columns={
        "short_geoid": "geoid",
        "total_count": "total_cra_loans",
        "total_amt": "total_cra_loan_amount"
    })
).to_sql("cra_fix", db, schema="meep", index=False, if_exists="replace")


-1

In [ ]:
# I didn't run this with the notebook originally, so it could have
# errors. (MV)

query = text("""
CREATE TABLE meep.small_business_bu AS TABLE meep.tableau_small_business;

UPDATE meep.tableau_small_business tsb
SET 
    total_cra_loans = cf.total_cra_loans,
    total_cra_loan_amount = cf.total_cra_loan_amount
FROM meep.cra_fix cf
WHERE tsb.geoid = cf.geoid;
""")

with db.connect() as conn:
    conn.execute(query)
    # conn.commit() # Do I need this? IDK